# F-тест: Совместная значимость

Тестируется гипотеза $H_0:\beta_1=\cdots=\beta_J=0$

Тестовая статистика $F$ (расчитывается автоматически)

Критическое значение $F_{cr}=F_{df1=J, df2=n-k-1}(\alpha)$

Гипотеза отвергается если $F>F_{cr}$ или $P<\alpha$

# LPM-модель: F-тест на совместную значимость

In [14]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col, summary_params # вывод результатов тестирования
from scipy.stats import f # F-распределение

## approve equation #1

Для датасета `loanapp` рассморим регрессию **approve на appinc, appinc^2, mortno, unem, dep, male, married, yjob, self**

In [15]:
# загрузим данные
loanapp_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))

In [16]:
#зададим спецификацию модели через формулу
mod_lpm = smf.ols(formula='approve~appinc+I(appinc**2)+mortno+unem+dep+male+married+yjob+self', data=loanapp_df)

In [17]:
# подгонка модели с поправкой на гетероскедастичность
res_lpm_hc = mod_lpm.fit(cov_type='HC3')
print(res_lpm_hc.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                approve   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.021
No. Observations:                1971   F-statistic:                     5.032
Covariance Type:                  HC3   Prob (F-statistic):           9.81e-07
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8420      0.027     31.003      0.000       0.789       0.895
appinc             0.0005      0.000      1.958      0.050   -4.48e-07       0.001
I(appinc ** 2) -1.007e-06   4.24e-07     -2.374      0.018   -1.84e-06   -1.76e-07
mortno             0.0660      0.015      4.321      0.000       0.036       0.096
unem              -0.0061      0.004     -1.515      0.130      -0.014       0.002
dep               -0.017

## Гипотеза 1

### Тестируем значимость влияния дохода, т.е. гипотезу $$H_0:\beta_{appinc}=\beta_{appinc^2}=0$$

### Степени свободы для нахождения критического значения: df1=2, df2=n-k-1

In [18]:
# зададим уровень значимости
alpha = 0.05

In [19]:
# 5%-критическое значение F-распределения
f.ppf(q=1-alpha, dfn=2, dfd=res_lpm_hc.df_resid).round(3)

3.0

## Результаты робастного F-теста

### Первый способ: через спецификацию гипотезы

In [20]:
# тестовая статистика, P-значение и степени свободы
print(res_lpm_hc.f_test('appinc=I(appinc ** 2)=0'))

<F test: F=2.960755420317824, p=0.05201130430276352, df_denom=1.96e+03, df_num=2>


In [21]:
# тестовая статистика, P-значение и степени свободы
print(res_lpm_hc.wald_test('appinc=I(appinc ** 2)=0', use_f=True))

<F test: F=array([[2.96075542]]), p=0.05201130430276352, df_denom=1.96e+03, df_num=2>


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


### Второй способ: через вспомогательную регрессию с ограничениями 
### `(не работает в случае робастной оценки ковариационной матрицы)`

In [22]:
# подгонка вспомогательной модели
restr1_hc = smf.ols(formula='approve~mortno+unem+dep+male+married+yjob+self', data=loanapp_df).fit(cov_type='HC3')

In [23]:
# тестовая статистика, P-значение и число ограничений
res_lpm_hc.compare_f_test(restr1_hc)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:2273: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  warnings.warn('F test for comparison is likely invalid with ' +


(5.148746797768447, 0.005885429662337571, 2.0)

## Результаты неробастного F-теста

### Первый способ: через спецификацию гипотезы

In [24]:
# подгонка модели
res_lpm_ols = mod_lpm.fit(cov_type='nonrobust')
# тестовая статистика, P-значение и степени свободы
print(res_lpm_ols.f_test('appinc=I(appinc ** 2)=0'))

<F test: F=5.148746799299675, p=0.005885429653372957, df_denom=1.96e+03, df_num=2>


### Второй способ: через вспомогательную регрессию с ограничениями

In [25]:
# подгонка вспомогательной модели
restr1_ols = smf.ols(formula='approve~mortno+unem+dep+male+married+yjob+self', data=loanapp_df).fit(cov_type='nonrobust')
# тестовая статистика, P-значение и число ограничений
res_lpm_ols.compare_f_test(restr1_ols)

(5.148746797768447, 0.005885429662337571, 2.0)

## **Вывод:** Гипотеза 1 не отвергается!